In [1]:
from keras.models import load_model, Model, Sequential
from keras.utils import plot_model
from keras.applications.resnet50 import ResNet50
from keras.layers import Input, Conv2D, Dropout, Dense, Flatten, AveragePooling2D, BatchNormalization, Activation
from keras.regularizers import l2
from keras.layers.merge import add, multiply

Using TensorFlow backend.


In [43]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    second_input = Input(shape=(175, 175, 3))
    semi_model = Model(input=parent.layers[0].input, outputs=parent.layers[-2].output)
    s = semi_model(second_input)
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    x = Flatten()(x)
    x = Dense(256)(x)
    x = multiply([semi_model.layers[-1].output, x])
    x = Dense(256)(x)
    x = Dropout(0.2)(x)
    y = Dense(256)(x) 
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=[inputs, second_input], outputs=y)
    
    gen = Model(inputs=[inputs, semi_model.layers[-1].output], outputs=y)
    return gen

In [3]:
parent = load_model('parent.h5')

In [44]:
gen = resnet_v1((175, 175, 3), 32, 108)

/home/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_2:0", shape=(?, 175, 175, 3), dtype=float32) at layer "input_2". The following previous layers were accessed without issue: ['input_19', 'conv2d_298', 'batch_normalization_280', 'activation_280', 'conv2d_299', 'batch_normalization_281', 'activation_281', 'conv2d_300', 'batch_normalization_282', 'add_136', 'activation_282', 'conv2d_301', 'batch_normalization_283', 'activation_283', 'conv2d_302', 'batch_normalization_284', 'add_137', 'activation_284', 'conv2d_303', 'batch_normalization_285', 'activation_285', 'conv2d_304', 'batch_normalization_286', 'add_138', 'activation_286', 'conv2d_305', 'batch_normalization_287', 'activation_287', 'conv2d_306', 'batch_normalization_288', 'add_139', 'activation_288', 'conv2d_307', 'batch_normalization_289', 'activation_289', 'conv2d_308', 'batch_normalization_290', 'add_140', 'activation_290', 'conv2d_309', 'batch_normalization_291', 'activation_291', 'conv2d_310', 'conv2d_3

In [40]:
parent.layers[1].layers[0]

In [15]:
semi_model = Model(input=parent.layers[0].input, outputs=parent.layers[-2].output)

/home/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [64]:
plot_model(resnet32, '')

In [32]:
semi_model.layers[0]

In [90]:
parent.layers[:]